# 🤖 AI Paragraph Generation (Ollama Cloud)

**목표**: Human 문단을 LLM(Ollama Cloud)에 넣어 AI 스타일 문단을 생성하여 (Human, AI) 페어 데이터셋 구축

## 🛠️ 설정
- **Model**: `gpt-oss:120b-cloud` 또는 `gpt-oss:20b-cloud`
- **API**: Ollama Cloud (https://ollama.com)
- **샘플 수**: 시작 500개 → 확장 가능

In [ ]:
# !pip install ollama pandas tqdm python-dotenv

In [38]:
import os
import pandas as pd
from tqdm.auto import tqdm
from ollama import Client
import time
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta

## 🔐 API 설정

In [39]:
load_dotenv()
API_KEY = os.environ.get('OLLAMA_API_KEY')
if not API_KEY:
    print("⚠️ OLLAMA_API_KEY가 설정되지 않았습니다.")
else:
    print("✅ API 키 로드 완료")

✅ API 키 로드 완료


In [46]:
if API_KEY:
    client = Client(
        host="https://ollama.com",
        headers={'Authorization': 'Bearer ' + API_KEY}
    )
    MODEL = 'gpt-oss:20b-cloud'
    print(f"🤖 사용 모델: {MODEL}")

🤖 사용 모델: gpt-oss:20b-cloud


## 📂 데이터 로드 및 샘플링

In [41]:
DATA_PATH = '/Users/youngjinson/Downloads/open/human_paragraphs.csv'
para_df = pd.read_csv(DATA_PATH)
print(f"총 문단 수: {len(para_df):,}개")

총 문단 수: 1,125,652개


In [42]:
N_DOCS = 100
SEED = 42

unique_doc_ids = para_df['source_doc_id'].unique()
sampled_doc_ids = pd.Series(unique_doc_ids).sample(n=min(N_DOCS, len(unique_doc_ids)), random_state=SEED).values
sampled_df = para_df[para_df['source_doc_id'].isin(sampled_doc_ids)].copy()
sampled_df = sampled_df.sort_values(['source_doc_id', 'para_index']).reset_index(drop=True)

print(f"샘플링: {len(sampled_doc_ids)}개 문서, {len(sampled_df)}개 문단")

샘플링: 100개 문서, 1174개 문단


## ✍️ 생성 함수

In [43]:
PROMPT_TEMPLATE = """다음 문단을 AI 스타일로 다시 작성해주세요. 
원본의 정보와 뜻은 유지하되, 자연스러운 AI 생성 텍스트처럼 작성해주세요. 종결어미는 원본을 따라해주세요.
추가 설명 없이 변환된 문단만 출력해주세요.

원본 문단:
{paragraph}

AI 스타일 문단:"""

def generate_ai_paragraph(paragraph, max_retries=3):
    prompt = PROMPT_TEMPLATE.format(paragraph=paragraph)
    for attempt in range(max_retries):
        try:
            response = client.chat(
                model=MODEL,
                messages=[{'role': 'user', 'content': prompt}],
                stream=False
            )
            return response['message']['content'].strip()
        except Exception as e:
            print(f"Error (attempt {attempt+1}): {e}")
            time.sleep(2 ** attempt)
    return None

## 🔄 1개씩 생성 + 저장 + 출력

In [44]:
OUTPUT_PATH = '/Users/youngjinson/Downloads/open/synthetic_pairs.csv'

# 기존 결과 로드 (이어서 하기 위함)
if os.path.exists(OUTPUT_PATH):
    result_df = pd.read_csv(OUTPUT_PATH)
    results = result_df.to_dict('records')
    completed_ids = set(result_df['para_id'].tolist())
    print(f"📂 기존 데이터 로드: {len(results)}개 완료")
else:
    results = []
    completed_ids = set()
    print("🆕 새로 시작")

print(f"남은 작업: {len(sampled_df) - len(completed_ids)}개")

🆕 새로 시작
남은 작업: 1174개


In [48]:
batch_start_time = time.time()
times_per_item = []

for idx, row in sampled_df.iterrows():
    # 이미 완료된 건 스킵
    if row['para_id'] in completed_ids:
        continue
    
    item_start = time.time()
    
    # 생성
    ai_paragraph = generate_ai_paragraph(row['paragraph'])
    
    item_elapsed = time.time() - item_start
    times_per_item.append(item_elapsed)
    
    # 결과 추가
    result = {
        'para_id': row['para_id'],
        'source_doc_id': row['source_doc_id'],
        'para_index': row['para_index'],
        'source_title': row['source_title'],
        'human_paragraph': row['paragraph'],
        'ai_paragraph': ai_paragraph,
        'generation_success': ai_paragraph is not None
    }
    results.append(result)
    completed_ids.add(row['para_id'])
    
    # 매번 저장
    pd.DataFrame(results).to_csv(OUTPUT_PATH, index=False)
    
    # 출력
    avg_time = sum(times_per_item) / len(times_per_item)
    remaining = len(sampled_df) - len(results)
    eta = str(timedelta(seconds=int(avg_time * remaining)))
    
    print(f"[{len(results)}/{len(sampled_df)}] {item_elapsed:.1f}초 | ETA: {eta}")
    print(f"  📝 {row['source_title'][:30]}...")
    print(f"  ✅ {'성공' if ai_paragraph else '실패'}")
    print()

total_elapsed = time.time() - batch_start_time
print(f"\n🎉 완료! 총 {len(results)}개, 소요시간: {str(timedelta(seconds=int(total_elapsed)))}")

[845/1174] 7.0초 | ETA: 0:38:20
  📝 심해탐사...
  ✅ 성공

[846/1174] 8.5초 | ETA: 0:42:16
  📝 심해탐사...
  ✅ 성공

[847/1174] 10.3초 | ETA: 0:46:44
  📝 심해탐사...
  ✅ 성공

[848/1174] 5.2초 | ETA: 0:42:02
  📝 영화...
  ✅ 성공

[849/1174] 12.6초 | ETA: 0:47:12
  📝 영화...
  ✅ 성공

[850/1174] 14.3초 | ETA: 0:52:05
  📝 영화...
  ✅ 성공

[851/1174] 44.8초 | ETA: 1:18:59
  📝 영화...
  ✅ 성공

[852/1174] 4.9초 | ETA: 1:12:10
  📝 영화...
  ✅ 성공

[853/1174] 7.0초 | ETA: 1:08:05
  📝 영화...
  ✅ 성공

[854/1174] 28.2초 | ETA: 1:16:06
  📝 영화...
  ✅ 성공

[855/1174] 2.9초 | ETA: 1:10:21
  📝 영화...
  ✅ 성공

[856/1174] 8.0초 | ETA: 1:07:48
  📝 영화...
  ✅ 성공

[857/1174] 15.7초 | ETA: 1:08:47
  📝 영화...
  ✅ 성공

[858/1174] 18.9초 | ETA: 1:10:46
  📝 영화...
  ✅ 성공

[859/1174] 10.0초 | ETA: 1:09:20
  📝 영화...
  ✅ 성공

[860/1174] 4.8초 | ETA: 1:06:21
  📝 영화...
  ✅ 성공

[861/1174] 3.6초 | ETA: 1:03:21
  📝 영화...
  ✅ 성공

[862/1174] 10.4초 | ETA: 1:02:38
  📝 영화...
  ✅ 성공

[863/1174] 5.5초 | ETA: 1:00:39
  📝 영화...
  ✅ 성공

[864/1174] 8.3초 | ETA: 0:59:35
  📝 영화...
  ✅ 성공

[865/